# 刘宇波老师《k 近邻算法》8 小节课笔记



# k 近邻（kNN）算法（1）基础知识

这一节简单介绍了 kNN 算法的基础知识和简单的实现，内容十分简单。

## 什么是 kNN

通过计算“待分类数据点”与“已有数据集”中的所有数据点的距离。取距离最小的前 $k$ 个点，根据“少数服从多数”的原则，将这个数据点划分为出现次数最多的那个类别。

具体的过程大致如下：

1. 计算待预测的数据点与训练数据集中所有已知点的**距离**，装入一个 List 容器中；
2. 对上面的 List 容器中的数据从小到大排列，取出其中的 $k$ 个；
3. 对其中的 $k$ 个进行投票，票数多的那个类别就是我们要预测的类别。

## 代码基本实现

### 第 1 步：计算待预测的数据点与训练数据集中所有已知点的**距离**，装入一个 List 容器中。

```python
from math import sqrt

distances = []
for x_train in X_train:
    # X 是我们要预测的那个点，例如 X = np.array([8.093607318,3.365731514])
    d = sqrt(np.sum((x_train - X)**2))
    distances.append(d)
distances
```

还可以这么写：

```
distances = [sqrt(np.sum((x_train - X)**2)) for x_train in X_train]
```

另外，NumPy 也提供了相应的方法用于计算距离：

```python
import numpy as np

distances = [np.linalg.norm(x_train - X) for x_train in X_train]
```

### 第 2 步：对上面的 List 容器中的数据从小到大排列，取出其中的 $k$ 个。

```python
# 按照距离从小到大排列
nearest = np.argsort(distances)
# 使用列表生成式，得到指定索引的标签值，然后进行投票
k = 6
topK_y = [y_train[i] for i in nearest[:k]]
```

### 第 3 步：对其中的 $k$ 个进行投票，票数多的那个类别就是我们要预测的类别。

```python
from collections import Counter

votes = Counter(topK_y)
# votes 长这样：Counter({0: 1, 1: 5})，表示 0 有 1 个，1 有 5 个

# votes.most_common(1) 的结果是 [(1, 5)]
# votes.most_common(2) 的解雇是 [(1, 5), (0, 1)]
# 所以我们对预测数据点 X 的预测结果就可以表示成如下
predict = votes.most_common(1)[0][0]
```
---
## 我的总结与思考

+ 虽说 kNN 我们在初次接触的时候，告诉我们它是一个分类算法，但 kNN 也可以用于回归，算法的思想非常简单，不过通过 kNN 的学习，可以让我们了解到机器学习算法的一般解决思路。
+ kNN 算法每一次的预测都要计算“预测数据点与训练数据集中的所有的点的距离”，计算量大。
+ 可以认为，训练数据集就是 kNN 算法本身，如果我们要使用这个算法，就要保存所有的训练数据集（耗费的内存多）。这一点不同于以后我们要学习的线性回归、逻辑回归、SVM ，这些算法学习到的是参数，可以这么认为，训练结束以后，我们得到了模型（算法）的参数，训练数据集就可以丢掉了。
  补充说明：这个例子就好像，我们推导数学公式一样，数学公式得到以后，我们以后再计算的时候套公式就可以了，我们不会像推导公式一样，去做一些重复的计算。
+ kNN 中的 $k$ 是一个超参数，所谓超参数，就是在模型训练之前，就要训练好的参数

---
## 参考资料

+ 可以在我的 Notebook 中查看所有的试验代码：[4-1 k 近邻算法基础.ipynb](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/4-1%20k%20%E8%BF%91%E9%82%BB%E7%AE%97%E6%B3%95%E5%9F%BA%E7%A1%80.ipynb)

---
## 待补充的内容

+ 自己实现的 kNN 算法的封装




# k 近邻（kNN）算法（2）在 scikit-learn 中使用 kNN

这一节介绍了如何在 scikit-learn 中使用 kNN 算法。

## 官方文档

+ 在官方文档中可以看到 ：可以使用 `KDTree` 和 `BallTree`  这两种数据结构来找到最近邻。我们上一节也提到过，我们最原始的通过计算距离的方式实现的 kNN 算法它的计算量很大，我们的目标仅仅是要找到距离预测点最近的那几个点，但是我们却把预测点与所有样本点的距离都计算了一遍。为此， `KDTree` 和 `BallTree` 就是两种高效的数据结构，来提高对预测点的最近 `k` 个邻居的搜索。
+ 除了超参数 `k` 以外，还可以指定超参数 `weights`。可选的参数值有 “uniform” 和 “distance”，“uniform” 的意思是每个近邻点的权重是一样的，通过“少数服从多数”，即“投票表决”的方式，选出类别最多的点作为待预测点的类别。如果使用 “distance” 作为 “weights” 的值，则表示距离越近的点，得到的权重值越大。
+ [手敲一下这个官方网站的例子](http://sklearn.apachecn.org/cn/0.19.0/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py)，[使用 kNN 算法预测鸢尾花数据集（官网例子，中文注释）](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/%E4%BD%BF%E7%94%A8%20kNN%20%E7%AE%97%E6%B3%95%E9%A2%84%E6%B5%8B%E9%B8%A2%E5%B0%BE%E8%8A%B1%E6%95%B0%E6%8D%AE%E9%9B%86%EF%BC%88%E5%AE%98%E7%BD%91%E4%BE%8B%E5%AD%90%EF%BC%89.ipynb)


## scikit-learn 的代码使用片段

```python
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# k 近邻算法也可以用于分类
# from sklearn.neighbors import KNeighborsRegressor
# 这里的 n_neighbors 是一个超参数

# k 近邻算法严重依赖训练数据集，可以认为训练数据集就是模型本身

# 原始的
raw_data_X = [[3.39353321, 2.33127338],
              [3.11007348, 1.78153964],
              [1.34380883, 3.36836095],
              [3.58229404, 4.67917911],
              [2.28036244, 2.86699026],
              [7.42343694, 4.69652288],
              [5.745052, 3.5339898],
              [9.17216862, 2.51110105],
              [7.79278348, 3.424088894],
              [7.93982082, 0.79163723]]
raw_data_y = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

X_train = np.array(raw_data_X)
y_train = np.array(raw_data_y)

kNN_classifier = KNeighborsClassifier(n_neighbors=6)
kNN_classifier.fit(X_train, y_train)

X = np.array([8.093607318, 3.365731514])
# 注意：这里传给 predict 方法的 dimension 应该是 2，所以，要 reshape 一下，reshape(-1, 2) 因为这里数据的维度是 2，或者 reshape(1, -1)
y_predict = kNN_classifier.predict(X.reshape(1, -1))
print(y_predict[0])
```

## 我的总结与思考

+ 查看官方文档

## 参考资料

+ [ApacheCN 组织翻译的官方文档](http://sklearn.apachecn.org/cn/0.19.0/modules/neighbors.html)
+ 可以在我的 Notebook 中查看所有的试验代码：[4-2 scikit-learn中的机器学习算法封装.ipynb](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/4-2%20scikit-learn%E4%B8%AD%E7%9A%84%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E7%AE%97%E6%B3%95%E5%B0%81%E8%A3%85.ipynb)



# k 近邻（kNN）算法（3）训练数据集与测试数据集
这一节介绍了为什么要在所有的训练数据集中分出一部分来用于测试。

## 划分测试数据集的意义
+ 如果训练好的模型没有测试就使用，我们就无从知道我们的模型是否有效。例如我们只做习题，没有模拟考试，就直接参加高考一样。
+ 真实环境中的数据的 label 或者说 target 有时候很难得到，所以从训练数据集中拿出一部分做预测是一种选择。
+ 给数据打上 label 或者说 target 有时工作量很大。

## 划分数据集的一个简单的实现

### 第 1 步：首先打乱数据，代码实现的时候，为了提高效率，我们不去移动数据，而选择打乱索引。

```python
# 为了复现，我们可以设置一个随机数的种子
# np.random.seed(666)
shuffle_indexes = np.random.permutation(len(X))
```


### 第 2 步：设置测试数据占全部数据的比例。
```python
# 分割出来的测试数据占未分割数据的比例
test_radio = 0.2
test_size = int(len(X) * test_radio)
```

### 第 3 步：最后分割得到训练数据集和测试数据集。

```python
test_indexes = shuffle_indexes[:test_size]
train_indexes = shuffle_indexes[test_size:]

X_train = X[train_indexes]
y_train = y[train_indexes]

X_test = X[test_indexes]
y_test = y[test_indexes]
```

可以将以上的步骤封装成一个方法来使用，不过 scikit-learn 已经为我们实现了。

## 使用 scikit-learn 中的 k 近邻算法完成训练-测试数据集的分割，并预测、计算准确度

```python
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 666)
```

---

## 参考资料

+ 可以在我的 Notebook 中查看所有的试验代码：[4-3 训练数据集，测试数据集.ipynb](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/4-3%20%E8%AE%AD%E7%BB%83%E6%95%B0%E6%8D%AE%E9%9B%86%EF%BC%8C%E6%B5%8B%E8%AF%95%E6%95%B0%E6%8D%AE%E9%9B%86.ipynb)​


# k 近邻（kNN）算法（4）分类准确度 accuracy

这一节介绍了评价分类好坏的标准之一：accuracy。

## 什么是分类准确度 accuracy

+ 评测模型在测试数据集上的效果，是一个量化的指标。
+ accuracy 其实很简单，这是对于分类问题而言的一个评判标准。accuracy = 分类正确的个数 / 总的数据个数。
+ 分类准确度涉及模型选择的问题。
+ 对于数据有极度偏斜的数据集来说，accuracy 不是一个很好的评判标准，这一点，我们以后再说。

我们使用 scikit-learn 为我们提供的手写数字识别的例子。
## 例子：手写数字识别

关键代码以及注释（完整请见：[ 4-4 分类准确度.ipynb](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/4-4%20%E5%88%86%E7%B1%BB%E5%87%86%E7%A1%AE%E5%BA%A6.ipynb)​
）：
```python
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# digit 手写数字数据集
digits = datasets.load_digits()

# 查看这个数据集的描述，从输出中可以找对这个数据集介绍的网页
digits.DESCR
# 查看这个数据集有哪些字段
digits.keys()

# 原始数据的特征矩阵
X = digits.data
# 原始数据的标签矩阵
y = digits.target

# 找特征矩阵的一行（即一个数据），看看到底是什么
some_digits = X[666]
some_digits_image = some_digits.reshape(8,8)
# imshow 看看数据是怎么回事
plt.imshow(some_digits_image, cmap = matplotlib.cm.binary)
plt.show()

# 计算分类准确度的步骤
# 1、训练数据集和测试数据集分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)
# 2、使用训练数据集训练模型
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
# 3、使用测试数据集预测
y_predict = knn_clf.predict(X_test)
# 4、计算分类准确度，下面 3 种方法都可以计算
accuracy1 = sum(y_predict == y_test) / len(y_test)
# 也可以使用 scikit-learn 提供的 sklearn.metrics.accuracy_score 方法直接计算
accuracy2 = accuracy_score(y_test, y_predict)
# 如果不关心预测的值是啥，可以直接把测试数据集的特征矩阵和标签矩阵传进去
accuracy3 = knn_clf.score(X_test, y_test)
```
---
## 参考资料

+ 可以在我的 Notebook 中查看所有的试验代码：[ 4-4 分类准确度.ipynb](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/4-4%20%E5%88%86%E7%B1%BB%E5%87%86%E7%A1%AE%E5%BA%A6.ipynb)​

# k 近邻（kNN）算法（5）超参数
这一节介绍了什么是机器学习算法的超参数。

## 什么是超参数，什么是模型参数
+ 超参数：在算法运行前需要确定的参数，kNN 算法中的 k 就是典型的超参数。
+ 模型参数：在算法执行的过程中学习的参数，kNN 算法没有模型参数。
+
## 寻找好的超参数的方法
+ 领域知识
+ 经验数值
+ 实验搜索：scikit-learn 就为我们提供了快捷的网格搜索的方式。

## 实验：直接写一个 for 循环，通过在测试数据集上的分类准确度 accuracy 来判断哪个 k 最好
下面的代码是一个很简单的实现了。

```python
best_score = 0.0
best_k = -1
for k in range(1,11):
    knn_clf = KNeighborsClassifier(n_neighbors=k)
    knn_clf.fit(X_train, y_train)
    score = knn_clf.score(X_test, y_test)
    if score > best_score:
        best_k = k
        best_score = score

print('best_k',best_k)
print('best_score',best_score)
```

## k 近邻算法的另一个超参数 weights

+ 如果我们希望离预测点较近的点，有更多的说服力说明是属于这些更近的点的类别，即越近的点有越大的权重，可以使用超参数 weights = 'distance' 来设置。
+ 如果设置了 weights = 'distance' 以后，如何定义距离，这又是一个可以讨论的话题，因此可以设置超参数 p 来修改距离的定义，这个 p 是 [Minkowski distance](https://en.wikipedia.org/wiki/Minkowski_distance) 定义的 p ，当 $p = 1$ 的时候，距离的定义是对应坐标绝对值之和， 当 $p = 2$ 的时候，距离的定义就是我们的欧氏距离（对应坐标的差的平方之和，再开根号）。

下面还是用粗暴的方法，写两个 for 循环来找到当 `weighs = 'distance'` 时候的最优超参数 `k` 和 `p`：

```
best_p = -1
best_score = 0.0
best_k = -1

for k in range(1,11):
    for p in range(1,6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights= 'distance', p = p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score
            best_p = p

print('best_p',best_p)
print('best_k',best_k)
print('best_score',best_score)
```
---
## 参考资料

+ Scikit-learn 官方文档：[Scikit-learn 官方文档中关于 sklearn.neighbors.KNeighborsClassifier 类的说明](http://sklearn.apachecn.org/cn/0.19.0/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)
+ 可以在我的 Notebook 中查看所有的试验代码：[4-5 超参数.ipynb](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/4-5%20%E8%B6%85%E5%8F%82%E6%95%B0.ipynb)​




# k 近邻（kNN）算法（6）使用 scikit-learn 提供的网格搜索来找到最优超参数"    ## 文章的标题

这一节介绍了什么是机器学习算法的超参数。

直接上代码说明问题：
```python
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# 网格搜索的第 1 步：超参数通过 json 字符串的方式传给 GridSearchCV
param_grid = [
    {
        'weights':['uniform'],
        'n_neighbors':[i for i in range(1, 11)]
    },
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1, 11)],
        'p':[i for i in range(1, 6)]

    }
]

digits = datasets.load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=666)

# 网格搜索的第 2 步：实例化 KNeighborsClassifier 的时候，不传任何超参数
knn_clf = KNeighborsClassifier()
# 网格搜索的第 3 步：实例化 GridSearchCV 类，传入模型和超参数
grid_search = GridSearchCV(knn_clf, param_grid)
# 网格搜索的第 4 步：使用 GridSearchCV 去训练
# 网格搜搜的计算量大，耗时多
grid_search.fit(X_train, y_train)

# 最佳 score
grid_search.best_score_
# 最佳的超参数
grid_search.best_params_
# {'n_neighbors': 3, 'p': 3, 'weights': 'distance'}

# estimator 评估量，网格搜索最佳分类器的超参数的值
grid_search.best_estimator_
# KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           # metric_params=None, n_jobs=1, n_neighbors=3, p=3,
           # weights='distance')

# 下面，我们就可以用那个最好的分类器，去运行我们的数据。
knn_clf = grid_search.best_estimator_
knn_clf.predict(X_test)
knn_clf.score(X_test, y_test)
```

更多 GridSearchCV 的参数的设置可以查看 Scikit-learn 官方文档：[Scikit-learn 官方文档中关于 sklearn.model_selection.GridSearchCV 类的说明](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)。

```python
# n_jobs = -1 将计算机所有的核都用于并行计算
# verbose 在搜索的过程中进行一些输出，整数值越大，输出的信息越详细，通常选择 2
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
```

---
## 参考资料

+ Scikit-learn 官方文档：[Scikit-learn 官方文档中关于 sklearn.model_selection.GridSearchCV 类的说明](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
+ 可以在我的 Notebook 中查看所有的试验代码：[ 4-6 网格搜索与k近邻算法中更多超参数.ipynb](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/4-6%20%E7%BD%91%E6%A0%BC%E6%90%9C%E7%B4%A2%E4%B8%8Ek%E8%BF%91%E9%82%BB%E7%AE%97%E6%B3%95%E4%B8%AD%E6%9B%B4%E5%A4%9A%E8%B6%85%E5%8F%82%E6%95%B0.ipynb)



# k 近邻（kNN）算法（7）Standardization

这一节介绍了如何在机器学习中做 Standardization。

要理解这一节的内容，我觉得主要把握以下几点，那就是：

+ 因为涉及到距离的概念，不同特征使用的单位不同，量纲不同，很可能导致在计算距离的时候，最终的距离被单位较大或者量纲较大的特征所主导
+ 那么如何解决这个问题呢：（1）将所有的特征缩放到一定区间（2）将所有的特征的均值处理成 0，方差处理成 1（这一点在本科的《概率论与数理统计》中有介绍），也叫数据的标准化。
+ 将所有的特征缩放到一定区间，这种方式的一个弊端是，如果数据集中有离群点的话，效果就不会很好。那么这种方法适用于什么情况呢？所有的数据有明显的边界，例如学生的考试成绩。
+ 一定要注意的一点是，**对于测试数据集，应该使用和训练数据集一样的均值、标准差（或者说最大值、最小值）进行处理，这一点一定要深刻体会它的必要性和合理性**。
+ 掌握这部分知识最好的学习方法是查看官方文档，根据不同 Standardization 的定义，自己实现一遍（套用公式，代码都很短），在使用 scikit-learn 提供的 API 计算一次。
+ 这一部分的知识，可以在官方文档的首页，在“数据的预处理”模块找到。

---
## 参考资料

+ 官方文档英文：[4.3. Preprocessing data](http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)
+ ApacheCN 翻译的官方文档中文版：[4.3. 预处理数据](http://sklearn.apachecn.org/cn/0.19.0/modules/preprocessing.html#preprocessing)
+ 可以在我的 Notebook 中查看所有的试验代码：[ 4-7 数据归一化.ipynb](https://nbviewer.jupyter.org/github/liweiwei1419/python-notes/blob/master/machine-learning-notes/%E7%AC%AC%204%20%E7%AB%A0%20%E6%9C%80%E5%9F%BA%E7%A1%80%E7%9A%84%E5%88%86%E7%B1%BB%E7%AE%97%E6%B3%95%20kNN/4-7%20%E6%95%B0%E6%8D%AE%E5%BD%92%E4%B8%80%E5%8C%96.ipynb)




# k 近邻（kNN）算法（8）更多关于 k 近邻算法的思考

这一节介绍了更多关于 k 近邻算法的思考。


更多关于 k 近邻算法的思考

+ k 近邻算法如何解决回归问题：找到最近的 k 个节点，取它们的平均值（或者加权平均）
+ k 近邻算法的效率低下
+ k 近邻算法高度数据相关
+ k 近邻算法预测的结果不具有可解释性
+ 维数灾难：所谓的维数灾难，就是“看似相近”的两个点之间的距离越来越大。（最好配图来说明）


机器学习的流程回顾
1、split
2、保证数据的特征在同一个尺度下
3、得到分类的准确度
4、为了获得最好的超参数，可以使用网格搜索的方法
